# Training notebook

This notebook will take you through all the steps to train a CNN classifier, the first step in a CODEX analysis. Follow the cells in order, modify them where needed and run them all. Hints and defaults values are suggested. Follow the training curves with Tensorboard to check whether you are satisfied with the model (good classification performance, moderated overfitting). You will probably have to give it a couple of runs before being fully satisfied. Keep in mind that the training will be orders of magnitude faster on GPU compared to CPU. If Pytorch and CUDA were properly set, the notebook will run on GPU without additional input from you.

## Import libraries

Make sure to import model class, dataset class and the required preprocessing operations (RandomCrop, ToTensor, Subtract...).

In [ ]:
# Standard libraries
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
import datetime
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import os
import zipfile
import time
import seaborn as sns
import pandas as pd
import warnings
import sys

# Custom functions/classes
path_to_module = '../source'  # Path where all the .py files are, relative to the notebook folder
sys.path.append(path_to_module)
from load_data import DataProcesser
from train_utils import accuracy, AverageMeter
from models import ConvNetCam, ConvNetCamBi
from models_lightning import LitConvNetCam, LitConvNetCamBi
from class_dataset import myDataset, ToTensor, Subtract, RandomShift, RandomNoise, RandomCrop, FixedCrop

# For reproducibility
myseed = 7
pl.utilities.seed.seed_everything(myseed)

## Define model architecture and set hyperparameters for training

Models are defined in the file `models.py`. There are already 2 models defined in this file: ConvNetCam and ConvNetCamBi. Both are variations of the CNN architecture, the former is for univariate input, the second for bivariate input.

Hyperparameters for training:
- nepochs: int, number of training epochs.
- batch_size: int, number of samples per batch.
- lr: float, initial learning rate. This is the most important parameter to tweak to have a smooth learning curve. 1e-2 is usually a good starting value.
- L2_reg: float, L2 regularization factor. This helps to prevent overfitting by penalizing large weights in the model parameters. In general try to always have a mild regularization, say 1e-3. Increase if you face overfitting issues.

By default, the learning rate is scheduled to decrease by a factor gamma at fixed epochs.
- lr_decrease_schedule: list, epochs at which to decrease.
- lr_decrease_factor: float, factor by which the lr is multiplied at the specified epochs.

Hyperparameters to set the model dimensions:
- length: int, length of a trajectory. This is the input length as the model will expect it. Setting it to a smaller value than the actual length of the trajectories can be used for data augmentation (see RandomCrop).
- nclass: int, number of output classes.
- nfeatures: int, size of the input representation before the output layer. This also corresponds to the number of filters in the last convolution layer.

In [ ]:
# Dimensions of the model
length = 220
nclass = 13
nfeatures = 10

# Optimization parameters
nepochs = 300
batch_size = 60
lr = 1e-2
# By default, use a lr scheduler which will decrease the lr at fixed epochs by a factor
lr_decrease_schedule = [75, 150, 250]
lr_decrease_factor = 0.1
L2_reg = 1e-3

# Model definition
#model = LitConvNetCam(batch_size=batch_size, lr_scheduler_milestones=lr_decrease_schedule, lr_gamma=lr_decrease_factor, nclass=nclass, length=length, nfeatures=nfeatures, lr=lr, L2_reg=L2_reg)  # for univariate input

model = LitConvNetCamBi(batch_size=batch_size, lr_scheduler_milestones=lr_decrease_schedule, lr_gamma=lr_decrease_factor, nclass=nclass, length=length, nfeatures=nfeatures, lr=lr, L2_reg=L2_reg)  # for univariate input

ngpu = -1  # number of GPUs to use, -1 means use all, 0 means use CPU

model.double()

## Load and process data, Data augmentation

Define which data to load and whether/how to preprocess the batch. 
- data_file: str, path to a .zip that can be loaded as a DataProcesser. The zip archive must contain 3 files: one for the
 data (dataset.csv), one for the split train/validation/test (id_set.csv), one with the classes informations (classes.csv). See DataProcesser.read_archive() for format details.
- meas_var: list of str or None, names of the measurement variables. In DataProcesser convention, this is the prefix in a
 column name that contains a measurement (time being the suffix). Pay attention to the order since this is how the dimensions of a sample of data will be ordered (i.e. 1st in the list will form 1st row of measurements in the sample, 2nd is the 2nd, etc...) If None, DataProcesser will extract automatically the measurement names and use the order of appearance in the column names.
- start_time/end_time: int or None, use to subset data to a specific time range. If None, DataProcesser will extract automatically the range of time in the dataset columns. Useful to completely exclude some acquisition times where irrelevant measures are acquired.

In [ ]:
# data_file = '../sample_data/Synthetic_Univariate.zip'
# data_file = '../sample_data/GrowthFactor_ErkAkt_Bivariate.zip'
data_file = '/home/marc/Dropbox/ERK_p21_Albeck/data_forCNN/erk_p21_EtopEGF_noInsulin.zip'

meas_var = None
start_time = None
end_time = None

In [ ]:
data = DataProcesser(data_file)

# Select measurements and times, subset classes and split the dataset
meas_var = data.detect_groups_times()['groups'] if meas_var is None else meas_var
start_time = data.detect_groups_times()['times'][0] if start_time is None else start_time
end_time = data.detect_groups_times()['times'][1] if end_time is None else end_time
data.subset(sel_groups=meas_var, start_time=start_time, end_time=end_time)
data.get_stats()
data.split_sets()

# Input preprocessing, this is done sequentially, on the fly when the input is passed to the network
average_perChannel = [data.stats['mu'][meas]['train'] for meas in meas_var]
ls_transforms = transforms.Compose([
    RandomCrop(output_size=length, ignore_na_tails=True),
    Subtract(average_perChannel),
    ToTensor()])

# Define the dataset objects that associate data to preprocessing and define the content of a batch
# A batch of myDataset contains: the trajectories, the trajectories identifier and the trajecotires class identifier
data_train = myDataset(dataset=data.train_set, transform=ls_transforms)
data_validation = myDataset(dataset=data.validation_set, transform=ls_transforms)

if batch_size > len(data_train) or batch_size > len(data_validation):
    raise ValueError('Batch size ({}) must be smaller than the number of trajectories in the training ({}) and the validation ({}) sets.'.format(batch_size, len(data_train), len(data_validation)))

# Quick recap of the data content
print('Channels order: {} \nTime range: ({}, {}) \nClasses: {}'.format(meas_var, start_time, end_time, list(data.dataset[data.col_class].unique())))
nclass_data = len(list(data.dataset[data.col_class].unique()))
if nclass != nclass_data:
    warnings.warn('The number of classes in the model output ({}) is not equal to the number of classes in the data ({}).'.format(nclass, nclass_data))

Plot some trajectories to check that the data loading and preprocessing is properly done. The curves appear here as they will be presented to the CNN.

In [ ]:
n_smpl = 6
indx_smpl = np.random.randint(0, len(data_train), n_smpl)

col_ids = []
col_lab = []
col_mes = []
# Long format for seaborn grid, for loop to avoid multiple indexing
# This would triggers preprocessing multiple times and add randomness
for i in indx_smpl:
    smpl = data_train[i]
    col_ids.append(smpl['identifier'])
    col_lab.append(smpl['label'].item())
    col_mes.append(smpl['series'].numpy().transpose())
col_ids = pd.Series(np.hstack(np.repeat(col_ids, length)))
col_lab = pd.Series(np.hstack(np.repeat(col_lab, length)))
col_mes = pd.DataFrame(np.vstack(col_mes), columns=meas_var)
col_tim = pd.Series(np.tile(np.arange(0, length), n_smpl))

df_smpl = pd.concat([col_ids, col_lab, col_tim, col_mes], axis=1)
df_smpl.rename(columns={0: 'identifier', 1: 'label', 2:'time'}, inplace=True)
df_smpl = df_smpl.melt(id_vars=['identifier', 'label', 'time'], value_vars=meas_var)

sns.set_style('white')
sns.set_context('notebook')
grid = sns.FacetGrid(data=df_smpl, col='identifier', col_wrap=3, sharex=True)
grid.map_dataframe(sns.lineplot, x='time', y='value', hue='variable')
grid.set(xlabel='Time', ylabel='Measurement Value')
grid.add_legend()

## Tensorboard logs and model save file

Define where the directory where training logs will be saved and the file where the model will be saved. By default, creates two subdirectories in the working directory: models and logs. An unique name for the logs and the model is created by appending the measurement name to the current timestamp.

The model training can be monitored in real-time in Tensorboard with these logs.

In [ ]:
file_data = os.path.splitext(os.path.basename(data_file))[0]  # file name without extension
timestamp = datetime.datetime.now().strftime('%Y-%m-%d-%H__%M__%S')

dir_logs = 'logs/'
subdir_logs = '_'.join(meas_var)
file_logs =  '_'.join([timestamp, file_data])
dir_model = 'models/' + '_'.join(meas_var)
file_model = '_'.join([timestamp, file_data])

if not os.path.exists(dir_model):
    os.makedirs(dir_model)
if not os.path.exists(dir_logs):
    os.makedirs(dir_logs)

tb_logger = pl.loggers.TensorBoardLogger(save_dir=dir_logs, name=subdir_logs, version=file_logs, default_hp_metric=False)
#chckpt_callback = ModelCheckpoint(dirpath=dir_model, filename=file_model)

## Train the model

Can follow the training in tensorboard with:
```
tensorboard --logdir "path/to/logs"
```

In [ ]:
train_loader = DataLoader(dataset=data_train,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4,
                          drop_last=True)

validation_loader = DataLoader(dataset=data_validation,
                               batch_size=batch_size,
                               shuffle=False,
                               num_workers=4,
                               drop_last=True)


#from torchsampler import ImbalancedDatasetSampler

#def custom_get_label(dataset, idx):
#    #callback function used in imbalanced dataset loader.
#    return int(dataset[idx]['label'])

#train_loader = DataLoader(dataset=data_train,
#                          batch_size=batch_size,
#                          sampler=ImbalancedDatasetSampler(data_train, callback_get_label=custom_get_label),
#                          num_workers=4,
#                          drop_last=True)


t0 = time.time()
lr_monitor = LearningRateMonitor(logging_interval='epoch')
trainer = pl.Trainer(callbacks=[
     lr_monitor,
     #chckpt_callback
     ],
 max_epochs=nepochs,
  min_epochs=nepochs,
   gpus=ngpu,
    log_every_n_steps=1,
     logger=tb_logger)
trainer.fit(model, train_loader, validation_loader)
t1 = time.time()

print('Elapsed time: {:.2f} min'.format((t1 - t0)/60))

## What's next

- Make sure that you are satisfied with the model. A quick way to do this is to check the Tensorboard logs. Check in priority if the classification performance are good enough (top1/top2 indicators) and that there is no serious problems of overfitting (difference between the training and validation sets should remain reasonably small).
- When possible, try to play around with the parameters before deciding on a given model. In priority, try to tweak the initial learning rate by a few orders of magnitude and the learning rate scheduler to see if you can get a smoothly converging training curve. If your model is overfitting try to reduce the number of features or to increase the L2 regularization strength.
- When you are satisfied with the model, you can go on the next notebooks to identify prototype trajectories and to identify class-discriminative motifs with CAMs.
- Alternatively, you can use the companion app to browse an interactive projection of the CNN features.

## Details and tips to setup loss and optimizer

The loss (called criterion in pytorch convention) against which the model is trained is defined here. For regular classification tasks use the CrossEntropyLoss. All available options are listed here: https://pytorch.org/docs/1.1.0/nn.html#loss-functions

The optimizer defines which algorithm is used to update the weights of the model through the training. L2 regularization is controlled by the "weight_decay" argument in the optimizer object. Adam optimizer with default parameters is a good default. All available options are listed here: https://pytorch.org/docs/1.1.0/optim.html#algorithms

Scheduler defines how the learning rate should evolve through the training. In order to get a smooth training and better classification performance, it needs to be reduced after some epochs. One of the easiest scheduler is the MultiStepLR which reduces the learning rate by a factor gamma when milestone epochs are reached. To start, just try to divide the learning rate by a factor 0.1, 3-4 times through the training. All available options are listed here: https://pytorch.org/docs/1.1.0/optim.html#how-to-adjust-learning-rate